In [14]:

import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]
SERVICE_ACCOUNT = SVC_ACC


PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

UID = UNIQUE_PREFIX

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}

!mkdir content
!rm -r content/clips/
!mkdir content/clips/
!rm -r content/frames/
!mkdir content/frames/

Updated property [core/project].
Creating gs://injae-sandbox-340804-flux-us-central1/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.
mkdir: cannot create directory ‘content’: File exists


In [15]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [16]:
# The pre-built serving docker image. It contains serving scripts and models.

SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240605_1400_RC00"

In [17]:
import base64
import glob
import io
import os
from datetime import datetime
from io import BytesIO

import imageio
from diffusers.utils import load_image
from google.cloud import aiplatform, storage
from IPython.display import HTML


def load_and_resize_image(image_url):
    image = load_image(image_url)
    # required by the model
    new_width = round(image.size[0] / 8) * 8
    new_height = round(image.size[1] / 8) * 8
    return image.resize((new_width, new_height))


def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def display_video_frames(frames, fps=7, width=500):
    io_obj = io.BytesIO()
    imageio.mimsave(io_obj, frames, format="mp4")
    video_data = base64.b64encode(io_obj.getvalue()).decode("utf-8")
    html = ""
    html += f"<video controls width={width}>"
    html += f'<source src="data:video/mp4;base64,{video_data}" type="video/mp4">'
    html += "</video>"
    display(HTML(html))


def upload_model(model_id, task):
    model_name = model_id
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    return model


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://<bucket-name>/<blob-name>
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))


def download_gcs_blob_as_json(gcs_file_path):
    """Download GCS blob and convert it to json."""
    client = storage.Client()
    bucket_name, blob_name = get_bucket_and_blob_name(gcs_file_path)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    return json.loads(blob.download_as_bytes())

## Image to Video

In [18]:
# @title Define model and task
model_id = "stabilityai/stable-video-diffusion-img2vid-xt"
task = "image-to-video"

In [27]:
import imageio

image = load_and_resize_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png"
)

# Create a list of frames by repeating the same image
frames = [image] * 5  # Repeat the image 5 times

# Save the frames as an MP4 video
imageio.mimsave("rocket_video.mp4", frames, format="mp4")

In [28]:
# @title Run inferences locally
import torch
from diffusers import StableVideoDiffusionPipeline

pipe = StableVideoDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()

# Load the conditioning image
image = load_and_resize_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png"
)

generator = torch.manual_seed(42)
frames_list = pipe([image], decode_chunk_size=8, generator=generator).frames



Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
# frames_list

from IPython.display import display

# Assuming frames_list is a list containing a single list of PIL Images
frames = frames_list[0] 

# for frame in frames:
#   display(frame)

from PIL import Image
def image_grid2(imgs, max_cols=5):
    num_imgs = len(imgs)
    cols = min(num_imgs, max_cols)
    rows = (num_imgs - 1) // cols + 1  # Calculate rows based on images and cols
    
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
grid2 = image_grid2(frames_list[0])
grid2

In [ ]:
from diffusers.utils import load_image, export_to_gif
export_to_gif(frames, "./main.gif")

In [ ]:
from IPython.display import Image

Image(url='./main.gif') 

## Final one

In [ ]:

# Load the conditioning image
image = load_and_resize_image('./tulip.jpeg'
)

generator = torch.manual_seed(42)
frames_list = pipe([image], decode_chunk_size=8, generator=generator).frames

# frames_list

from IPython.display import display

# Assuming frames_list is a list containing a single list of PIL Images
frames = frames_list[0] 

# for frame in frames:
#   display(frame)
from PIL import Image


grid2 = image_grid2(frames)
grid2



In [ ]:


from diffusers.utils import load_image, export_to_gif
export_to_gif(frames[10:], "./tulip_main.gif")

from IPython.display import Image

Image(url='./tulip_main.gif') 